In [17]:
import tensorflow as tf
# import tensorflow_datasets as tfdt
from keras import layers
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BertTokenizer
import numpy as np

In [18]:
# Read Dataset
data = pd.read_csv('./dataset/dataset.csv')

data = data.sample(frac=1).reset_index(drop=True)


In [21]:
# Split the Dataset

train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))

print('Train Size : ', train_size)
print('Val Size : ', val_size)

train_df = data[:train_size]
val_df = data[train_size:train_size+val_size]
test_df = data[train_size+val_size:]

print(f'Train Length : {len(train_df)}')
print(f'Val Length : {len(val_df)}')
print(f'Test Length : {len(test_df)}')

Train Size :  96000
Val Size :  12000
Train Length : 96000
Val Length : 12000
Test Length : 12000


In [28]:
text_col = 'description'
label_col = 'label'
class_names = ['World', 'Sports', 'Business', 'Sci/Tech']

def subtract_one_label(text, label):
  return text,label -1

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Pake model pre trained jadi langsung jadi 

# Convert dataframe (training, validation, and testing) ke dalam tensorflow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df[text_col].values, train_df[label_col].values))
train_data = train_data.map(subtract_one_label)
print(train_data)

val_data = tf.data.Dataset.from_tensor_slices((val_df[text_col].values, val_df[label_col].values))
val_data = val_data.map(subtract_one_label)
print(val_data)

test_data = tf.data.Dataset.from_tensor_slices((test_df[text_col].values, test_df[label_col].values))
test_data = test_data.map(subtract_one_label)
print(test_data)

<_MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
<_MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [29]:
# Pre Processing

BUFFER_SIZE = 10000
BATCH_SIZE = 64
MAX_LENGTH = 256

def encode_text(text, label):
  encoded_text = bert_tokenizer.encode(text.numpy().decode('utf-8', add_special_tokens=True, max_length=MAX_LENGTH, truncation=True, padding='max-length'))
  return encoded_text, label

train_data = train_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))
val_data = val_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))
test_data = test_data.map(lambda text, label: tf.py_function(encode_text, [text, label], [tf.int64, tf.int64]))

# Pad dataset and batch dataset
